**Company** : <br>
Design Firm

**Notebook Function** : <br>
    This notebook processes the identification measures 

**Output File(s)** : <br>
    hash_embeddings_50_all_quarterly_internal.csv - A file containing the final identification measures for each person-quarter
    
**Author(s)** : <br>
Lara Yang, Sarayu Anshuman

Install packages and import libraries

In [ ]:
pip install mittens

In [ ]:
pip install gensim

In [ ]:
import os
import sys
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
from mittens import Mittens
import csv
from operator import itemgetter
import json
import ujson
import re
from gensim.matutils import cossim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from statistics import mean 
import multiprocessing
from hashlib import md5
from collections import Counter
import random
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import random
from datetime import datetime, timezone
import us
import pytz
num_cores = multiprocessing.cpu_count()
num_users_to_test = 12

year_colname, quarter_colname, halfyear_colname, yearmonth_colname = 'year', 'quarter', 'halfyear', 'yearmonth'
num_cores = 12

Run helper function for calculating th eidentification measures for each person-quarter

In [ ]:
#########################################################################
########### Helper Functions for Generating Mittens Embeddings ##########
#########################################################################

def _window_based_iterator(toks, window_size, weighting_function):
    for i, w in enumerate(toks):
        yield w, w, 1
        left = max([0, i-window_size])
        for x in range(left, i):
            yield w, toks[x],weighting_function(x)
        right = min([i+1+window_size, len(toks)])
        for x in range(i+1, right):
            yield w, toks[x], weighting_function(x)
    return

def glove2dict(glove_filename):
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        data = {line[0]: np.array(list(map(float, line[1: ]))) for line in reader}
    return data

# Inspired by original build_weighted_matrix in utils.py in the Mittens paper source codebase
def build_weighted_matrix(files,
        mincount=300, vocab_size=None, window_size=10,
        weighting_function=lambda x: 1 / (x + 1), load_to_mem=False, parallel=False, verbose=False,
        internal_only=False):
    """
    Builds a count matrix based on a co-occurrence window of
    `window_size` elements before and `window_size` elements after the
    focal word, where the counts are weighted based on proximity to the
    focal word.
    Parameters
    ----------
    files : list of strings
        Filenames where documents in a corpus are located
    corpus : iterable of str
        Texts to tokenize.
    mincount : int
        Only words with at least this many tokens will be included.
    vocab_size : int or None
        If this is an int above 0, then, the top `vocab_size` words
        by frequency are included in the matrix, and `mincount`
        is ignored.
    window_size : int
        Size of the window before and after. (So the total window size
        is 2 times this value, with the focal word at the center.)
    weighting_function : function from ints to floats
        How to weight counts based on distance. The default is 1/d
        where d is the distance in words.
    load_to_mem : bool, optional
        Whether to load entire corpus to memory
    parallel : bool, optional
        Whether to process emails in parallel. Only makes sense if loading to memory.
    internal_only : bool, optional
        Whether to only include internal emails - emails whose recipients only include company employees
    Returns
    -------
    pd.DataFrame
        This is guaranteed to be a symmetric matrix, because of the
        way the counts are collected.
    """
    wc = defaultdict(int)
    # This variable will be a list when load_to_mem is true and a generator when load_to_mem is false
    corpus = None
    if load_to_mem:
        if parallel:
            pool = multiprocessing.Pool(processes = num_cores)
            results = [pool.apply_async(read_corpus_parallel, args=(f, internal_only, )) for f in files]
            corpus = [r.get() for r in results if r.get()]
            pool.close()
            pool.join()
        else:
            # previously used split(' ') in 2yp
            corpus = [text.split() for text in read_corpus(files, internal_only)]
        for toks in corpus:
            for tok in toks:
                wc[tok] += 1
        sys.stderr.write('\n')
    else:
        corpus = read_corpus(files, internal_only)
        for text in corpus:
            for tok in text.split():
                wc[tok] += 1
    if verbose: sys.stderr.write('Finished counting %d unique words in corpus at %s.\n' % (len(wc), datetime.now()))
    if vocab_size:
        srt = sorted(wc.items(), key=itemgetter(1), reverse=True)
        vocab_set = {w for w, c in srt[: vocab_size]}
    else:
        vocab_set = {w for w, c in wc.items() if c >= mincount}
    vocab = sorted(vocab_set)
    n_words = len(vocab)
    if verbose: sys.stderr.write('Finished generating vocab of %d words at %s.\n' % (n_words, datetime.now()))
    # Weighted counts:
    # Generator function needs to be re-initated
    if not load_to_mem: corpus = read_corpus(files, internal_only)
    counts = defaultdict(float)
    for toks in corpus:
        if not load_to_mem: toks = toks.split()
        window_iter = _window_based_iterator(toks, window_size, weighting_function)
        for w, w_c, val in window_iter:
            if w in vocab_set and w_c in vocab_set:
                counts[(w, w_c)] += val
    if verbose: sys.stderr.write('Finished counting co-occurrences across %d word pairs at %s.\n' % (len(counts), datetime.now()))
    X = np.zeros((n_words, n_words))
    for i, w1 in enumerate(vocab):
        for j, w2 in enumerate(vocab):
            X[i, j] = counts[(w1, w2)]
    if verbose: sys.stderr.write('Finished converting co-occurrences to sorted matrix of shape %s at %s.\n' % (str(X.shape), datetime.now()))
    X = pd.DataFrame(X, columns=vocab, index=pd.Index(vocab))
    return X

# this method is currently unused as it was used for proof-of-concept during macro lunch,
# where only employees with the top 200 vocab counts are included
def select_usr_corpus(inpath, num_usrs_to_process):
    infile = pd.read_csv(inpath)
    usr2num_tokens = infile.set_index(["usr"]).dropna().to_dict()["num_tokens"]
    usrs_to_analyze = [usr for usr, num_tokens in sorted(usr2num_tokens.items(), key=lambda item: item[1])]
    usrs_to_analyze = usrs_to_analyze[-num_usrs_to_process:]
    return usrs_to_analyze  

# The function currently ignores sentence delimiters and considers co-occurrences across sentence boundaries to be
# co-occurrences as well. If desired, we can vary yield behavior based on the sentence_delim boolean to yield every
# sentence instead of every message as it currently does. However, if we were to yield every sentence, we need to make
# sure to not simply output the body but to yield every new line in the body individually.
def read_corpus(files, internal_only, sentence_delim=False):
    for file in files:
        with open(file, errors='ignore', encoding='utf-8') as infile:
            try:
                msg = ujson.load(infile)
                if internal_only and not is_internal_msg(msg):
                    continue
                # This line ignores all sentence structures as we are using \n to represent sentence splits
                body = msg['hashed-body'].replace('\n', ' ').strip()
                if len(body) > 0:
                    yield body
                else:
                    continue
            except (ValueError, json.decoder.JSONDecodeError) as error:
                continue
    return

def read_corpus_parallel(f, internal_only, sentence_delim=False):
    with open(f, errors='ignore', encoding='utf-8') as infile:
        try:
            msg = ujson.load(infile)
            if internal_only and not is_internal_msg(msg):
                return None
            # This line ignores all sentence structures as we are using \n to represent sentence splits
            body = msg['hashed-body'].replace('\n', ' ').strip()
            if len(body) > 0:
                return body.split()
            else:
                return None
        except (ValueError, json.decoder.JSONDecodeError) as error:
            return None

def output_embeddings(mittens_df, filename, compress=False):
    """
    Writes embeddings in the format of pd.DataFrame to a text file, replacing any existing file with the
    same name
    """
    if compress:
        mittens_df.to_csv(filename + '.gz', quoting=csv.QUOTE_NONE, header=False, sep=" ", compression='gzip')
    else:
        mittens_df.to_csv(filename, quoting=csv.QUOTE_NONE, header=False, sep=" ")
    return

#########################################################################
############# Helper Functions for Working with JSON Emails #############
#########################################################################
# Same as get_recipient function in Analysis/acculturaltion/lingdistance/jensen_shannon.py
def get_recipients(msg):
    sender = msg['from'][0] if type(msg['from']) == list else msg['from']
    rec = set(msg['to'] + msg['cc'] + msg['bcc']) - set([sender])
    return rec

def is_internal_msg(msg):
    """
    Determines whether msg is an internal message. An internal message is a message that is only sent to internal employees,
    whose anon IDs start with 'U'. An individual who is not an internal employee does not have a domain associated with design firm and
    their anon IDs start with 'E'.
    """
    for r in get_recipients(msg):
        if re.match('E---', r):
            return False
    return True

def slice_user_corpus(files, train_mode):
    timekey2files = defaultdict(list)
    #the function iterates through each email in the email list
    for file in files:
        with open(file, errors='ignore') as infile:
            try:
                msg = ujson.load(infile)
                if train_mode == 'annual':
                    timekey2files[to_year(msg['date'], format='str')].append(file)
                elif train_mode == 'quarterly':
                    timekey2files[to_quarter(msg['date'], format='str')].append(file)
                elif train_mode == 'monthly':
                    timekey2files[to_yearmonth(msg['date'], format='str')].append(file)
                elif train_mode == 'halfyear':
                    timekey2files[to_halfyear(msg['date'], format='str')].append(file)
                elif train_mode == 'all':
                    timekey2files[to_year(msg['date'], format='str')].append(file)
                    timekey2files[to_quarter(msg['date'], format='str')].append(file)
                    timekey2files[to_halfyear(msg['date'], format='str')].append(file)
                    #timekey2files[to_yearmonth(msg['date'], format='str')].append(file) #removed as it leads to too many output files
            except (ValueError, json.decoder.JSONDecodeError) as error:
                continue
    return timekey2files

def read_message(files):
    errors = 0
    for file in files:
        with open(file, errors='ignore') as infile:
            try:
                msg = ujson.load(infile)
                yield msg
            except (ValueError, json.decoder.JSONDecodeError) as error:
                errors += 1
                continue
    sys.stderr.write("{} files produced errors out of a total of {} files.\n".format(errors, len(files)))
    return

#########################################################################
############# Helper Functions for Working with Date Objects ############
#########################################################################

def to_yearmonth(date, format):
    if format == 'str':
        return date[0:7]
    elif format == 'datetime':
        return date.strftime('%Y-%m')

def to_quarter(date, format):
    year, month = 0, 0
    if format == 'str':
        year = date[0:4]
        month = date[5:7]    
    elif format == 'datetime':
        year = date.year
        month = date.month
    quarter = ((int(month)-1) // 3) + 1
    timekey = str(year) + 'Q' + str(quarter)
    return timekey

def to_halfyear(date, format):
    """
    Return half year of date in string
    """
    year, month = 0, 0
    if format == 'str':
        year = date[0:4]
        month = date[5:7]    
    elif format == 'datetime':
        year = date.year
        month = date.month
    halfyear = ((int(month)-1) // 6) + 1
    timekey = str(year) + 'HY' + str(halfyear)
    return timekey

def to_year(date, format):
    if format == 'str':
        return date[0:4]
    elif format == 'datetime':
        return str(date.year)

def datetime_to_timekey(date, time_key):
    if time_key == 'year':
        return to_year(date, format='datetime')
    elif time_key == 'quarter':
        return to_quarter(date, format='datetime')
    elif time_key == 'halfyear': #added this new
        return to_halfyear(date, format='datetime') #added this new
    elif time_key == 'yearmonth':
        return to_yearmonth(date, format='datetime')

def is_month_before_equal(datetime1, datetime2):
    if datetime1.year < datetime2.year:
        return 1
    elif (datetime1.year == datetime2.year) and (datetime1.month <= datetime2.month):
        return 1
    else:
        return 0

def num_months_between_dates(datetime1, datetime2):
    return abs((datetime1.year - datetime2.year) * 12 + datetime1.month - datetime2.month)

def num_quarters_between_dates(datetime1, datetime2):
    return abs((datetime1.year - datetime2.year) * 12 + datetime1.month - datetime2.month) // 3

def num_years_between_dates(datetime1, datetime2):
    return abs(datetime1.year - datetime2.year)

def time_between_dates(datetime1, datetime2, time_key):
    if time_key == 'monthly':
        return num_months_between_dates(datetime1, datetime2)
    elif time_key == 'quarterly':
        return num_quarters_between_dates(datetime1, datetime2)
    elif time_key == 'annual':
        return num_years_between_dates(datetime1, datetime2)

#########################################################################
############## Helper Functions for Working with Dataframes #############
#########################################################################
def dict_to_df(index2rows, cols, index_name):
    """
    Parameters
    ----------
    index2rows : dict
        Dictionary mapping index to rows to be coverted
    cols : list
        List of column names of type str
    index : list
        List of index names
    Returns
    -------
    df : pd.DataFrame
        Constructed dataframe
    """
    if len(index_name) == 1:
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df.index.name = index_name[0]
        df.sort_index(axis=0, inplace=True)
        return df
    else:
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df = pd.DataFrame(df, pd.MultiIndex.from_tuples(df.index, names=index_name))
        df.sort_index(axis=0, inplace=True)
        return df

def generate_test_df():
    """
    A function to generate a DataFrame for testing purposes.

    This DataFrame uses Gender as the index and has a column for simulated word vectors for 'i'
    and a column for simulated word vectors for 'we'. Each of these word vectors is a size-10 numpy vector.
    """
    df = pd.DataFrame()
    df['gender'] = ['F', 'M', 'M', 'F', 'M', 'F', 'F', 'M', 'M', 'F']
    df['i'] = df.apply(lambda x: np.random.rand(10).round(5), axis=1)
    df['we'] = df.apply(lambda x: np.random.rand(10).round(5), axis=1)
    df.set_index('gender', inplace=True)
    return df

#########################################################################
########### Helper Functions for Working with Embedding Output ##########
#########################################################################

def remove_empty_embeddings(embeddings_dir):
    """
    Removes all empty files in embeddings_dir that were produced when vocab size was 0.
    Parameters
    ----------
    embeddings_dir : str
        Full path to directory where embedding files are located
    """
    for file in os.listdir(embeddings_dir):
        mittens_file = os.path.join(embeddings_dir, file)
        if os.path.getsize(mittens_file) == 0:
            os.remove(mittens_file)
    return

def word_similarity(model, w1, w2):
    """
    This is an auxilary function that allows for comparing one word to another word or multiple words
    If w1 and w2 are both single words, n_similarity returns their cosine similarity which is the same as 
    simply calling similarity(w1, w2)
    If w1 or w2 is a set of words, n_similarity essentially takes the mean of the set of words and then computes
    the cosine similarity between that vector mean and the other vector. This functionality is both reflected
    in its source code and has been verified manually.
    Parameters
    ----------
    model : KeyedVectors
        The model that contains all the words and vectors
    w1 : str or list
        The first word or word list to be compared
    w2 : str or list
        The second word or word list to be compared
    Returns
    -------
    float
        Cosine similarity between w1 and w2
    """
    if not isinstance(w1, list):
        w1 = [w1]
    if not isinstance(w2, list):
        w2 = [w2]
    #w1 = [w for w in w1 if w in model.vocab]
    w1 = [w for w in w1 if w in model.key_to_index]
    #w2 = [w for w in w2 if w in model.vocab]
    w2 = [w for w in w2 if w in model.key_to_index]
    if len(w1) == 0 or len(w2) == 0:
        return None
    return model.n_similarity(w1, w2)

def extract_company_embedding(company_embeddings_filename, tmp_dir, words):
    """
    Parameters
    ----------
    company_embeddings_filename : str
        File path of the company embeddings
    tmp_dir : str
        Path to the directory for gensim to output its tmp files in order to load embeddings into word2vec format
    words : list
        A list of strings to retrieve vectors for
    Returns
    -------
    vecs : list
        A list of vectors of type numpy.ndarray that correspond to the list of words given as parameters
    """ 
    tmp_mittens = os.path.join(tmp_dir, "mittens_embeddings_all_word2vec.txt")
    word2vec_mittens_file = get_tmpfile(tmp_mittens)
    glove2word2vec(company_embeddings_filename, word2vec_mittens_file)
    model = KeyedVectors.load_word2vec_format(word2vec_mittens_file)
    vecs = []

    '''
    for w in words:
        if w in model.vocab:
            vecs.append(model.wv[w])
        else:
            print('%s not in company embeddings' % w)
    '''
    for w in words:
     try:
      index = model.key_to_index[w]
      vecs.append(model.vectors[index])
     except KeyError:
      vecs.append(np.nan)
      print('%s not in company embeddings' % w)

    return vecs

def isnull_wrapper(x):
    r = pd.isnull(x)
    if type(r) == bool:
        return r
    return r.any()

def cossim_with_none(vec1, vec2, vec_format='sparse'):
    """
    Auxiliary function that calls cossim function to test if vectors are None to prevent erroring out.
    Parameters
    ----------
    vec1 : list of (int, float), gensim sparse vector format
    vec2 : list of (int, float), gensim sparse vector format
    format : str, optional
        Either sparse or dense. If sparse, vec1 and vec2 are in gensim sparse vector format; use cossim function from gensim.
        Otherwise, vec1 and vec2 are numpy arrays and cosine similarity is hand calculated
    Returns
    -------
    float
        Cosine similarity between vec1 and vec2
    """
    if not (isnull_wrapper(vec1) or isnull_wrapper(vec2)):
        if vec_format == 'sparse':
            return cossim(vec1, vec2)
        elif vec_format == 'dense':
            if len(vec1) == 0 or len(vec2) == 0:
                return None
            return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))
        else:
            raise ValueError()
    return None

def calculate_pairwise_cossim(col1, col2=None, reference=False, reference_group=None, anon_ids=None):
    """
    Calculates averaged cosine similarity of every row vector in col1 with every other row vector in col2.
    If no col2 is provided, cosine similarity of every row vector with every other row vector in col1 is calculated.
    The two columns should have equal length.
    Parameters
    ----------
    col1 : pd.Series
        A column where each row is a sparse word vector (BoW format that gensim code is written for).
    col2 : pd.Series, optional
        A column where each row is a sparse word vector (BoW format that gensim code is written for).
    reference : bool, optional
        Indicator variable for whether filtering for reference groups is needed
    reference_group : pd.Series, optional
        If filtering for reference groups, a list containing reference group members for every employee in col1
    anon_ids : pd.Series, optional
        If filtering for reference groups, a list containing anon_ids for every employee in col1
    Returns
    -------
    results : list
        A list where the ith element is the averaged cosine similarity between the ith vector in col1 and every vector
        in col2 for which i != j. If no col2 is provided, a list where the ith element is the averaged cosine similarity
        between the ith vector in col1 and every other vector in col1 is returned.
    """
    vectors1 = col1.tolist()
    vectors2 = col2.tolist() if col2 is not None else col1.tolist()
    reference_group = reference_group.tolist() if reference else None
    anon_ids = anon_ids.tolist() if anon_ids is not None else None
    results = list()
    for i in range(len(vectors1)):
        total_sim = []
        if vectors1[i]:
            for j in range(len(vectors2)):
                if i != j and vectors2[j]:
                    # filter out any np.nans as our reference group
                    if not reference or (type(reference_group[i]) == set and anon_ids[j] in reference_group[i]):
                        total_sim.append(cossim(vectors1[i], vectors2[j]))
            results.append(mean(total_sim) if len(total_sim) > 0 else None)
        else:
            results.append(None)
    return results

def vector_mean(col):
    """
    Calculate vector means of row vectors
    Parameters
    ----------
    col : pd.Series
        The column to be averaged
    Returns
    -------
    np.array
        A vector that is the numerical average of all vectors in col
    """
    return np.array(col[col.notna()].tolist()).mean(axis=0)

def fix_department(hr_df):
    """
    Correcting all typos in the department field. Mostly for generating a correct department
    roster used by mittens3d_explore_alternative_embeddings.py
    Parameters
    ----------
    hr_df : pd.DataFrame
        A dataframe of HR data
    Returns
    -------
    hr_df : pd.DataFrame
        The original dataframe after typos are fixed in the department column
    """
    hr_df['department'] = hr_df['department'].str.replace("Enviroments", "Environments", regex=True)
    hr_df['department'] = hr_df['department'].str.replace("Leanrnign", "Learning", regex=True)
    hr_df['department'] = hr_df['department'].str.replace(" +$", "", regex=True)
    return hr_df

def convert_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, format='%m-%d-%y')
    except ValueError:
        return pd.to_datetime(date_str, format='%m-%d-%Y')

def extract_hr_df(hr_filepath, time_key=None):
    hr = pd.read_csv(hr_filepath)
    cols = hr.columns
    cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)
    hr.columns = cols
    #hr['snapshot_date'] = pd.to_datetime(hr['snapshot_date'],format='%m-%d-%Y')
    hr['snapshot_date'] = hr['snapshot_date'].apply(convert_to_datetime)
    hr = hr.sort_values(by=['anon_id', 'snapshot_date'])
    # drop all rehires
    hr_rehires = set(hr.loc[hr['rehire_date'].notna(), 'anon_id'])
    hr['drop'] = hr.apply(lambda row : 1 if row.anon_id in hr_rehires else 0, axis=1)
    hr = hr.loc[hr['drop'] == 0]
    hr['hire_date'] = pd.to_datetime(hr['hire_date'], 'coerce')
    hr['termination_date'] = pd.to_datetime(hr['termination_date'])
    hr['termination_reason'] = hr['termination_reason'].str.lower()
    hr.dropna(subset=['hire_date', 'snapshot_date'], axis=0, inplace=True)
    hr['entry_age'] = hr.groupby('anon_id')['age'].transform('min')
    hr['exit'] = hr.apply(lambda row : 1 if pd.notna(row['termination_reason']) else 0, axis=1)
    hr['exit_vol'] = hr.apply(lambda row : 1 if (row['termination_reason'] == 'voluntary') else 0, axis=1)
    hr['exit_invol'] = hr.apply(lambda row : 1 if (row['termination_reason'] == 'involuntary') else 0, axis=1)
    hr = fix_department(hr)
    if not time_key: #person level
        supervisors = set(hr['supervisor_id'])
        hr['is_supervisor'] = hr.apply(lambda row : 1 if row.anon_id in supervisors else 0, axis=1)
        hr['initial_salary'] = hr.apply(lambda row: row.annual_salary if (row.snapshot_date.month == row.hire_date.month) else None, axis=1)
        hr['initial_salary'].fillna(method = 'ffill', inplace=True)
        hr['salary_delta'] = (hr['annual_salary'] - hr['initial_salary']) / hr['initial_salary']
        hr['salary_avg'] = hr.groupby('anon_id')['annual_salary'].transform('mean')
        hr['tenure'] = hr.apply(lambda row:
            (row.termination_date - row.hire_date).days if pd.notna(row.termination_reason) else (row.snapshot_date - row.hire_date).days, axis=1)
        hr.drop_duplicates(subset='anon_id', keep='last', inplace=True)
        hr = (hr[['anon_id', 'gender', 'ethnicity', 'department', 'location', 'job_title',
            'tenure', 'exit', 'exit_vol', 'exit_invol', 'is_supervisor', 'salary_delta', 'salary_avg', 'entry_age']])
        hr.set_index('anon_id', inplace=True)
        hr.index.name = 'usr'
    else:
        hr[time_key] = hr.apply(lambda row : datetime_to_timekey(row['snapshot_date'], time_key), axis=1)
        hr['past_salary'] = hr.groupby('anon_id')['annual_salary'].shift(1)
        hr['past_job_title'] = hr.groupby('anon_id')['job_title'].shift(1)
        hr['promotion'] = hr.apply(lambda row : 1 if (row['annual_salary'] > row['past_salary'] and (row['past_job_title'] != row['job_title'])) else 0, axis=1)
        monthly_supervisors = {k: set(sups['supervisor_id'].tolist()) for k, sups in hr[['snapshot_date', 'supervisor_id']].groupby("snapshot_date")}
        hr['is_supervisor'] = hr.apply(lambda row : 1 if row['anon_id'] in monthly_supervisors[row['snapshot_date']] else 0, axis=1)
        if time_key == quarter_colname or time_key == year_colname or time_key == halfyear_colname:
            hr['promotion'] = hr.groupby(['anon_id', time_key])['promotion'].transform('max')
            hr['exit'] = hr.groupby(['anon_id', time_key])['exit'].transform('max')
            hr['exit_vol'] = hr.groupby(['anon_id', time_key])['exit_vol'].transform('max')
            hr['exit_invol'] = hr.groupby(['anon_id', time_key])['exit_invol'].transform('max')
            hr['is_supervisor'] = hr.groupby(['anon_id', time_key])['is_supervisor'].transform('max')
            hr.drop_duplicates(subset=['anon_id', time_key], keep='last', inplace=True)
        hr = (hr[['anon_id', time_key, 'snapshot_date', 'gender', 'ethnicity', 'department', 'location', 'job_title', 'annual_salary', 'hire_date',
                'is_supervisor', 'age', 'entry_age', 'promotion', 'exit', 'exit_vol', 'exit_invol']])
        hr.set_index(['anon_id', time_key], inplace=True)
    return hr

Now run a function to calculate the hash for the design firm

In [1]:
def cheap_hash(to_hash, n=8): 
    """
    Computes a shortened version of the hash of an input string.
    Reduces storage space; defense against lookup tables
    Collision risk checked - no collision found using 20,000 most common English words
    Parameters
    ----------
    to_hash : str
        A string to be hashed
    Returns
    -------
    hashed : str
        A string that represents the hash of to_hash with length n
    """
    hashed = md5(to_hash.encode('utf-8')).hexdigest()[:n] 
    return hashed

Identifify the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Set the hyperparameters and input and output directories

In [ ]:
internal_only = True
first_batches = False
num_cores = 14 
num_users_to_test = 12
mincount = 50

#set directory paths
home_dir = current_dir
home_dir1 = "/zfs/projects/faculty/amirgo-transfer/design/"
corpus_dirs = [os.path.join(home_dir1, "hashed_corpus"), os.path.join(home_dir1, "hashed_corpus_batch3")]
utils_dir = os.path.join(home_dir, "utils")
embeddings_dir="/zfs/projects/faculty/amirgo-identification/identification-Sarayu/design/fine-tuning/hash_embeddings_50_all_internal"
tmp_dir = os.path.join(current_dir, "tmp")
output_dir = os.path.join(current_dir, "idtf_output_data")

#name final output file
quarterly_output_filename = os.path.join(output_dir, "hash_embeddings_{}_b12_quarterly".format(mincount) if first_batches else "hash_embeddings_{}_all_quarterly".format(mincount))
if internal_only: quarterly_output_filename += '_internal'
quarterly_output_filename += '.csv'

#read the hr file for the design firm
hr_filepath = os.path.join(home_dir,"anonymized_hr_data.csv")
quarter_colname = 'quarter'
pronouns = ['i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves']
single_pronouns = ['i', 'we']
hash_pronouns = [cheap_hash(p) for p in pronouns]
hash_single_pronouns = [cheap_hash(p) for p in single_pronouns]
i_index, we_index = 0, 5

print(corpus_dirs)
print(embeddings_dir)
print(tmp_dir)
print(output_dir)
print(quarterly_output_filename)
print(hr_filepath)

Run helper functions to generate the identification measures

In [ ]:
#########################################################################
############### Functions for Calculating Frequency Counts ##############
#########################################################################
def scale(x):
    """
    This scale function provides the same result as R's scale. sklearn's preprocessing.scale is slightly different as it uses N instead of N-1
    in calculating standard deviation 
    """
    return (x-np.mean(x))/np.std(x, ddof=1)

def reading_message_time(i, num_users, usr_dirs, usr):
    msg2time = {}
    files = [os.path.join(usr_dir, file) for usr_dir in usr_dirs for file in os.listdir(usr_dir)]    
    for msg in read_message(files):
        if msg['date'] is not None and msg['time'] is not None:
            msg2time[msg['message-id']] = msg['date'] + ' ' + msg['time']
    return msg2time

def reading_usr_counts(i, num_users, usr_dirs, usr, msg2time, usr_quarter2timezone, slice_type):
    """
    Reading raw counts for a specific user
    Parameters
    ----------
    i : int
        Index of current user in all users; used for keeping track of progress
    num_users : int
        Total number of users; used to keeping track of progress
    usr_dirs : list of str
        Contains full path directories that contain emails of the focal user
    usr : str
        Focal user whose emails we are currently processing; used for keeping track of progress
    msg2time : dict
        Maps unique message IDs to when they were sent; used for measuring average response time of emails
    usr_quarter2timezone: dict
        Maps user and quarter to timezone in which they work to provide additional precision to calculating effort variables
        based on local timezones of user
    slice_type : str
        Either 'quarterly' or 'halfyearly'
    Returns
    -------
    quarter2var2counts : dict
        A dictionary mapping quarters to names of variables of interest to their values
    """

    files = [os.path.join(usr_dir, file) for usr_dir in usr_dirs for file in os.listdir(usr_dir)]
    sys.stderr.write("Processing \t%d/%d -'%s' with %d files, at %s.\n" % (i, num_users, usr, len(files), datetime.now()))
    quarterly2counts = defaultdict(lambda : [0, 0, 0, 0, set(), set(), []])
    pacific = pytz.timezone('America/Los_Angeles')
    for msg in read_message(files):
        date = msg['date']
        if date is None:
            continue
        if internal_only and not is_internal_msg(msg):
            continue

        if slice_type == 'quarterly':
            quarter = to_quarter(date, format="str")
        else: #this is the 'halfyearly case'
            quarter = to_halfyear(date, format="str") #the variable quarter would actually contain the halfyear information in this case
        
        date = datetime.strptime(date + ' ' + msg['time'], '%Y-%m-%d %H:%M:%S')
        # dates have all been converted to Pacific Timezone in clean.py
        timezoned_date = pacific.localize(date).astimezone(pytz.timezone(usr_quarter2timezone[(usr, quarter)])) if (usr, quarter) in usr_quarter2timezone else date
        weekday = timezoned_date.weekday()
        hour = timezoned_date.hour

        quarterly2counts[quarter][0] += len(msg['hashed-body'].replace('\n', ' ').strip().split())
        quarterly2counts[quarter][1] += 1
        for r in get_recipients(msg):
            assert r != usr, "Created a self-loop"
            quarterly2counts[quarter][5].add(r)

        if (weekday == 5) or (weekday == 6):
            quarterly2counts[quarter][2] += 1
            quarterly2counts[quarter][4].add(timezoned_date.strftime('%Y-%m-%d'))
        elif (hour < 8) or (hour > 17):
            quarterly2counts[quarter][3] += 1
        if msg['in-reply-to'] is not None and msg['in-reply-to'] in msg2time:
            diff = date - datetime.strptime(msg2time[msg['in-reply-to']], '%Y-%m-%d %H:%M:%S')
            quarterly2counts[quarter][6].append(diff.total_seconds())
    for quarter in quarterly2counts.keys():
        quarterly2counts[quarter][4] = len(quarterly2counts[quarter][4])
        if len(quarterly2counts[quarter][6]) > 0:
            quarterly2counts[quarter][6] = sum(quarterly2counts[quarter][6])/len(quarterly2counts[quarter][6])
        else:
            quarterly2counts[quarter][6] = None
    return dict(quarterly2counts)

def reading_raw_counts(corpus_dirs, first_batches, hr_df_quarterly, slice_type='quarterly', test_mode=False):
    """
    The main workhorse function for obtaining raw message and token counts as control variables.
    Parameters
    ----------
    corpus_dirs : list of str
        Contains full path directories to both email corpora, batch 1 & 2 and batch 3
    first_batches : bool
        If true, only use emails in batch 1 and 2
    hr_df_quarterly : pd.DataFrame
        Used for performing departmental standardizing of effort variables, 
        but we can also pass a halfyearly file, so the local variable names in the function will still go by quarter in this case, but it will contain halfyearly data
    slice_type :
        Either 'quarterly' or 'halfyearly'
    test_mode : bool, optional
        If true, select a subset of user directories
    Returns
    -------
    usr_quarter2counts_df : pd.DataFrame
        Dataframe with user and quarter as multi-index and message and token counts as columns
    """
    if first_batches:
        corpus_dir = corpus_dirs[0]
        usr_dirs = [os.path.join(corpus_dir, usr_dir) for usr_dir in os.listdir(corpus_dir) if os.path.isdir(os.path.join(corpus_dir, usr_dir))]
    else:
        usr_dirs = [os.path.join(corpus_dir, usr_dir) for corpus_dir in corpus_dirs for usr_dir in os.listdir(corpus_dir) if os.path.isdir(os.path.join(corpus_dir, usr_dir))]

    if test_mode: usr_dirs = [usr_dirs[random.randint(0, len(usr_dirs)-1)] for _ in range(len(usr_dirs)//100)] 
    
    usr2dirs = defaultdict(list)
    for usr_dir in usr_dirs:
        usr2dirs[os.path.basename(usr_dir)].append(usr_dir)
    num_users = len(usr2dirs)

    sys.stderr.write('Generating msg:time dictionary for %d directories at %s.\n' % (len(usr_dirs), str(datetime.now())))
    msg2time = {}
    pool = multiprocessing.Pool(processes = num_cores)
    results = []
    for i, (usr, dirs) in enumerate(usr2dirs.items()):
        results.append(pool.apply_async(reading_message_time, args=(i, num_users, dirs, usr, )))
    pool.close()
    pool.join()
    for r in results:
        d = r.get()
        msg2time.update(d)
    
    usr_quarter2state = hr_df_quarterly['location'].to_dict()
    states = hr_df_quarterly['location'].to_list()

    #generating state timezone dictionary
    state2timezone = {}
    for state in set(states):
        processed_state = state.strip()
        if 'CA - ' in processed_state:
            processed_state = 'CA'
        elif 'Washington State' in processed_state:
            processed_state = 'Washington'
        us_state = us.states.lookup(processed_state)
        if us_state:
            # some states span more than one time zone; if so, choose the first one
            state2timezone[state] = us_state.time_zones[0]
        else:
            print('{} is not a valid U.S. state.'.format(processed_state))
    usr_quarter2timezone = {key:state2timezone[state] for key, state in usr_quarter2state.items()}
    
    sys.stderr.write('Reading all corpora to produce token and message counts at %s.\n' % str(datetime.now()))
    usr_quarter2counts = defaultdict(lambda : [0, 0, 0, 0, set(), set(), []])
    pool = multiprocessing.Pool(processes = num_cores)
    results = {}
    for i, (usr, dirs) in enumerate(usr2dirs.items()):
        results[usr] = pool.apply_async(reading_usr_counts, args=(i, num_users, dirs, usr, msg2time, usr_quarter2timezone, slice_type, ))
    pool.close()
    pool.join()
    for usr, r in results.items():
        quarterly2counts = r.get()
        for quarter in sorted(quarterly2counts.keys()):
            usr_quarter2counts[(usr, quarter)] = quarterly2counts[quarter]
    for usr, quarter in list(usr_quarter2counts):
        for peer in usr_quarter2counts[(usr, quarter)][5]:
            usr_quarter2counts[(peer, quarter)][5].add(usr)

    for usr, quarter in list(usr_quarter2counts):
        peers = usr_quarter2counts[(usr, quarter)][5]
        peer_num_messages_weekend = [usr_quarter2counts[(p, quarter)][2] for p in peers] + [usr_quarter2counts[(usr, quarter)][2]]
        peer_num_messages_post_work = [usr_quarter2counts[(p, quarter)][3] for p in peers] + [usr_quarter2counts[(usr, quarter)][3]]
        usr_working_weekends = usr_quarter2counts[(usr, quarter)][4] if type(usr_quarter2counts[(usr, quarter)][4]) == int else 0
        peer_num_working_weekends = [usr_quarter2counts[(p, quarter)][4] if type(usr_quarter2counts[(p, quarter)][4]) == int else 0 for p in peers] + [usr_working_weekends]
        usr_quarter2counts[(usr, quarter)].append(scale(peer_num_messages_weekend)[-1])
        usr_quarter2counts[(usr, quarter)].append(scale(peer_num_messages_post_work)[-1])
        usr_quarter2counts[(usr, quarter)].append(scale(peer_num_working_weekends)[-1])

    cols = (['num_tokens', 'num_messages', 'num_messages_weekend', 'num_messages_post_work', 'num_working_weekends', 'network_peers', 'avg_response_time',
    'peer_standardized_num_messages_weekend', 'peer_standardized_num_messages_post_work', 'peer_standardized_num_working_weekends'])
    
    if slice_type == 'quarterly':
        usr_quarter2counts_df = dict_to_df(usr_quarter2counts, cols, index_name=['anon_id', quarter_colname])
        usr_quarter2counts_df = usr_quarter2counts_df.join(hr_df_quarterly)
    
        #usr_quarter2counts_df['department_standardized_num_messages_weekend'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_messages_weekend'].apply(scale)
        usr_quarter2counts_df['department_standardized_num_messages_weekend'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_messages_weekend'].transform(scale)
        #usr_quarter2counts_df['department_standardized_num_messages_post_work'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_messages_post_work'].apply(scale)
        usr_quarter2counts_df['department_standardized_num_messages_post_work'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_messages_post_work'].transform(scale)
    
        #usr_quarter2counts_df['num_working_weekends'] = usr_quarter2counts_df['num_working_weekends'].apply(lambda x : x if type(x) == int else 0)
        usr_quarter2counts_df['num_working_weekends'] = usr_quarter2counts_df['num_working_weekends'].transform(lambda x: np.where(pd.api.types.is_integer_dtype(x), x, 0))
    
        #usr_quarter2counts_df['department_standardized_num_working_weekends'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_working_weekends'].apply(scale)
        usr_quarter2counts_df['department_standardized_num_working_weekends'] = usr_quarter2counts_df.groupby(['quarter', 'department'])['num_working_weekends'].transform(scale)
    
    else: #this is the case for halfyearly
        usr_quarter2counts_df = dict_to_df(usr_quarter2counts, cols, index_name=['anon_id', halfyear_colname])
        usr_quarter2counts_df = usr_quarter2counts_df.join(hr_df_quarterly)
        usr_quarter2counts_df['department_standardized_num_messages_weekend'] = usr_quarter2counts_df.groupby(['halfyear', 'department'])['num_messages_weekend'].transform(scale)
        usr_quarter2counts_df['department_standardized_num_messages_post_work'] = usr_quarter2counts_df.groupby(['halfyear', 'department'])['num_messages_post_work'].transform(scale)
        usr_quarter2counts_df['num_working_weekends'] = usr_quarter2counts_df['num_working_weekends'].transform(lambda x: np.where(pd.api.types.is_integer_dtype(x), x, 0))
        usr_quarter2counts_df['department_standardized_num_working_weekends'] = usr_quarter2counts_df.groupby(['halfyear', 'department'])['num_working_weekends'].transform(scale)
    
    return usr_quarter2counts_df

#########################################################################
#### Functions for Measuring Within-Person Similarities in Embeddings ###
#########################################################################

def embeddings_similarities(model):
    """
    Returns the cosine similarity between i and we, and the cosine similarity between i-words and we-words
    Parameters
    ----------
    model : gensim.models.Word2Vec
        Model that stores the embeddings for each word
    Returns
    -------
    embeds : list
        A list of cosine similarity between "i" and "we", i-words and we-words, and symmetric i-words and
        we-words, in that order; an i-word is symmetric if and only if its equivalent we-word is in the model vocab
    """
    embeds = ([word_similarity(model, hash_single_pronouns[0], hash_single_pronouns[1]),
        word_similarity(model, hash_pronouns[i_index:we_index], hash_pronouns[we_index:])])
    symmetric_i_words, symmetric_we_words = [], []
    for i in range(len(hash_pronouns)-we_index):
        #if hash_pronouns[i] in model.vocab and hash_pronouns[i+we_index] in model.vocab:
        if hash_pronouns[i] in model.key_to_index and hash_pronouns[i+we_index] in model.key_to_index:
            symmetric_i_words.append(hash_pronouns[i])
            symmetric_we_words.append(hash_pronouns[i+we_index])
    if len(symmetric_i_words) > 0:
        embeds.append(model.n_similarity(symmetric_i_words, symmetric_we_words))
    return embeds

#load all the embeddings into the tmp directory with the appropriate extension
def process_single_embedding_file(i, num_files, embeddings_dir, file):
    """
    Reading from one embedding file
    Parameters
    ----------
    i : int
        Index used for progress tracking
    num_files : int
        Total number of files to process used for progress tracking
    embedding_dir : str
        Directory in which embedding files reside
    file : str
        Embedding file to open and process
    Returns
    -------
    embeds : list
        A list of cosine similarity between "i" and "we", i-words and we-words, and symmetric i-words and
        we-words, in that order; an i-word is symmetric if and only if its equivalent we-word is in the model vocab
    """
    mittens_file = os.path.join(embeddings_dir, file)
    if i % 100 == 0:
        sys.stderr.write("Processing \t%d/%d -'%s', at %s.\n" % (i, num_files, mittens_file, datetime.now()))
    # chopping off the file extension in filename
    tmp_mittens = os.path.join(tmp_dir, file[0:-4] + "_word2vec.txt")
    try:
        word2vec_mittens_file = get_tmpfile(tmp_mittens)
        glove2word2vec(mittens_file, word2vec_mittens_file)
        model = KeyedVectors.load_word2vec_format(word2vec_mittens_file)
        embeds = embeddings_similarities(model)
        return embeds
    except Exception as e:
        sys.stderr.write('File %s caused an error: %s.\n' % (mittens_file, str(e)))

def extract_usr_timekey(file):
    """ 
    Extract timekey from file, with format hash_embedding_usr_timekey.txt
    """
    file_chunks = file[0:-4].split('_') #obtain the name of the file until '.txt'
    usr = file_chunks[2] #file is of the fome 'hash_embeddings_U0_2013HY1.txt'
    time_key = file_chunks[3] if len(file_chunks) == 4 else None
    return (usr, time_key)

def self_similarities(files, num_files, embeddings_dir):
    """
    Main workhorse function for calculating within-person similarities. Compares an individual's i embedding to we embedding, using both
    i and we's embedding only and centroid of i-words and we-words. Indices used in this file relies on knowledge of the naming convention of underlying embedding files.
    Parameters
    ----------
    files : list of str
        Embedding files to process
    num_files : int
        Total number of files to process, used to keep track of progress
    embeddings_dir : str
        Directory in which embedding files reside
    Returns
    -------
    usr_quarter2distances : dict
        Dictionary mapping usr and quarter to within-person embedding similarities
    """
    usr_quarter2distances = defaultdict(list)
    usr_halfyear2distances = defaultdict(list)
    pool = multiprocessing.Pool(processes = num_cores)
    results = {}
    for i, file in enumerate(files, 1):
        usr, time_key = extract_usr_timekey(file) #usr is U0, U1 and so on, timekey is either 2013 or 2013Q1 or 2013HY1
        if i % 100 == 0:
            print('time_key')
            print(time_key)
        results[(usr, time_key)] = pool.apply_async(process_single_embedding_file, args=(i, num_files, embeddings_dir, file, ))
    pool.close()
    pool.join()
    for key, r in results.items():
        usr, time_key = key
        curr_row = r.get()
        # Empty if errored out
        if curr_row:
            if time_key and len(time_key) == 6: #ex. 2020Q1
                    usr_quarter2distances[(usr, time_key)] = curr_row
            elif time_key and len(time_key) == 7: #ex. 2020HY1
                    usr_halfyear2distances[(usr, time_key)] = curr_row
            else:
                    sys.stderr.write('Embedding file format does not conform to expectations. Extracted time key %s for user %s.\n' % (time_key, usr)) 
    return (usr_quarter2distances, usr_halfyear2distances)

def reading_embeddings(embeddings_dir, test_mode=False):
    """
    Calculates embedding similarities within-person and between-person
    Parameters
    ----------
    embeddings_dir : str
        Directory where all embedding files exist
    test_mode : bool, optional
        If testing, reduce number of files to process
    Returns
    -------
    usr_quarter2distances_df : pd.DataFrame
        Panel data at the quarter level that includes i vs we and i-words vs we-words cosine similarity 
    """
    files = os.listdir(embeddings_dir)
    if test_mode: files = [files[random.randint(0, len(files)-1)] for _ in range(len(files)//50)]

    num_files = len(files)

    sys.stderr.write('Calculate within-person similarities for %d files at %s.\n' % (num_files, str(datetime.now())))
    usr_quarter2distances, usr_halfyear2distances = self_similarities(files, num_files, embeddings_dir)

    cols = ['i_we', 'i_we_cluster', 'i_we_symmetric']
    usr_quarter2distances_df = dict_to_df(usr_quarter2distances, cols, index_name=['anon_id', quarter_colname])
    usr_halfyear2distances_df = dict_to_df(usr_halfyear2distances, cols, index_name=['anon_id', halfyear_colname])

    return (usr_quarter2distances_df, usr_halfyear2distances_df)

In [ ]:
starttime = datetime.now()
test = False
for d in [output_dir, tmp_dir]:
    if not os.path.exists(d):
    	os.mkdir(d)

#for quarterly hr data
sys.stderr.write('Reading quarterly HR data at %s.\n' % datetime.now())           
hr_df_quarterly = extract_hr_df(hr_filepath, quarter_colname)
#for half yearly hr data 
sys.stderr.write('Reading halfyearly HR data at %s.\n' % datetime.now())           
hr_df_halfyearly = extract_hr_df(hr_filepath, halfyear_colname)
sys.stderr.write('Reading corpus at %s.\n' % str(datetime.now()))
usr_quarter2counts_df = reading_raw_counts(corpus_dirs, first_batches, hr_df_quarterly, 'quarterly', test)
sys.stderr.write('Reading corpus at %s.\n' % str(datetime.now()))
usr_halfyear2counts_df = reading_raw_counts(corpus_dirs, first_batches, hr_df_halfyearly, 'halfyearly', test)
sys.stderr.write('Reading embeddings at %s.\n' % datetime.now())
usr_quarter2distances_df,usr_halfyear2distances_df = reading_embeddings(embeddings_dir, test)
sys.stderr.write('Outputting dataframe at %s.\n' % datetime.now())
usr_quarter2counts_df.join(usr_quarter2distances_df).to_csv(quarterly_output_filename)
usr_halfyear2counts_df.join(usr_halfyear2distances_df).to_csv(halfyearly_output_filename)

sys.stderr.write("Finished outputting measures at %s, with a duration of %s.\n"
    % (str(datetime.now()), str(datetime.now() - starttime)))